# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nilayjhaveri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nilayjhaveri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [7]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/nilayjhaveri/Workspace/Projects/ai_maker_space/ai_engineering/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/nilayjhaveri/Workspace/Projects/ai_maker_space/ai_engineering/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/nilayjhaveri/Workspace/Projects/ai_maker_space/ai_engineering/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [12]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'headlines' already exists in node '0a6eb5'. Skipping!
Property 'headlines' already exists in node '88e1c2'. Skipping!
Property 'headlines' already exists in node '5cc3be'. Skipping!
Property 'headlines' already exists in node '560d4d'. Skipping!
Property 'headlines' already exists in node '4c4fb5'. Skipping!
Property 'headlines' already exists in node '4b6de0'. Skipping!
Property 'headlines' already exists in node 'd83507'. Skipping!
Property 'headlines' already exists in node 'fc81ce'. Skipping!
Property 'headlines' already exists in node '0af6fc'. Skipping!
Property 'headlines' already exists in node '2bd9be'. Skipping!
Property 'headlines' already exists in node 'f3310e'. Skipping!
Property 'headlines' already exists in node 'adfac5'. Skipping!
Property 'headlines' already exists in node 'f24c39'. Skipping!
Property 'headlines' already exists in node '4ea684'. Skipping!
Property 'headlines' already exists in node '8b7577'. Skipping!
Property 'headlines' already exists in n

Applying HeadlineSplitter:   0%|          | 0/40 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/59 [00:00<?, ?it/s]

...

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/83 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '88e1c2'. Skipping!
Property 'summary_embedding' already exists in node '0a6eb5'. Skipping!
Property 'summary_embedding' already exists in node '671737'. Skipping!
Property 'summary_embedding' already exists in node 'd83507'. Skipping!
Property 'summary_embedding' already exists in node 'f24c39'. Skipping!
Property 'summary_embedding' already exists in node '4ea684'. Skipping!
Property 'summary_embedding' already exists in node '0af6fc'. Skipping!
Property 'summary_embedding' already exists in node 'adfac5'. Skipping!
Property 'summary_embedding' already exists in node '5cc3be'. Skipping!
Property 'summary_embedding' already exists in node '2bd9be'. Skipping!
Property 'summary_embedding' already exists in node '8b7577'. Skipping!
Property 'summary_embedding' already exists in node '4c4fb5'. Skipping!
Property 'summary_embedding' already exists in node '4b6de0'. Skipping!
Property 'summary_embedding' already exists in node '560d4d'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 74, relationships: 2220)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 74, relationships: 2220)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
Review:

SingleHopSpecificQuerySynthesizer - This generates straightforward, direct queries that can be answered with a single step or "hop" through the data. These are specific, focused questions that don't require complex reasoning or multiple data points to answer.
MultiHopAbstractQuerySynthesizer - This creates more complex queries that require multiple steps or "hops" to answer, but the questions are abstract in nature. These queries might need to connect different pieces of information or require higher-level reasoning to resolve.
MultiHopSpecificQuerySynthesizer - This generates complex queries that require multiple steps to answer, but unlike the abstract version, these queries are specific and concrete. They need multiple data points or reasoning steps but ask for precise, specific information.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [19]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the purpose of the Knowledge Center?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not specify the purpose of th...,single_hop_specifc_query_synthesizer
1,Could you please explain the significance of 3...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,"In the context of Chapter 3, how does inclusio...",[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in academic ...,[Non-Term Characteristics A program that measu...,A program that measures progress in clock hour...,single_hop_specifc_query_synthesizer
4,What is the purpose of Appendix A?,[both the credit or clock hours and the weeks ...,Appendix A provides examples illustrating how ...,single_hop_specifc_query_synthesizer
5,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
6,Wht are the instrucional time and weeks of ins...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatons specify that for credit hour pr...,multi_hop_abstract_query_synthesizer
7,How do the regulatory citations in 34 CFR 668....,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatory citations in 34 CFR 668.3(a) an...,multi_hop_abstract_query_synthesizer
8,How does the inclusion of clinical work in a s...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"According to Volume 8, if clinical work meets ...",multi_hop_specific_query_synthesizer
9,How do the definitions of academic years in Vo...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 specifies that an academic year must ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c4e53f'. Skipping!
Property 'summary' already exists in node 'a42deb'. Skipping!
Property 'summary' already exists in node 'c51a37'. Skipping!
Property 'summary' already exists in node '2209e9'. Skipping!
Property 'summary' already exists in node 'c64367'. Skipping!
Property 'summary' already exists in node '3674dc'. Skipping!
Property 'summary' already exists in node 'ca83f0'. Skipping!
Property 'summary' already exists in node '2a5461'. Skipping!
Property 'summary' already exists in node '2af44a'. Skipping!
Property 'summary' already exists in node '9cb0fd'. Skipping!
Property 'summary' already exists in node 'fff614'. Skipping!
Property 'summary' already exists in node 'b524f2'. Skipping!
Property 'summary' already exists in node '1d5557'. Skipping!
Property 'summary' already exists in node '36f4b0'. Skipping!


...

Property 'summary_embedding' already exists in node 'c4e53f'. Skipping!
Property 'summary_embedding' already exists in node '2a5461'. Skipping!
Property 'summary_embedding' already exists in node 'ca83f0'. Skipping!
Property 'summary_embedding' already exists in node 'a42deb'. Skipping!
Property 'summary_embedding' already exists in node 'c51a37'. Skipping!
Property 'summary_embedding' already exists in node 'fff614'. Skipping!
Property 'summary_embedding' already exists in node '2af44a'. Skipping!
Property 'summary_embedding' already exists in node '3674dc'. Skipping!
Property 'summary_embedding' already exists in node '2209e9'. Skipping!
Property 'summary_embedding' already exists in node '9cb0fd'. Skipping!
Property 'summary_embedding' already exists in node 'b524f2'. Skipping!
Property 'summary_embedding' already exists in node '36f4b0'. Skipping!
Property 'summary_embedding' already exists in node 'c64367'. Skipping!
Property 'summary_embedding' already exists in node '1d5557'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are academic years in the context of educ...,"[Chapter 1 Academic Years, Academic Calendars,...",Academic years are defined as periods that inc...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about in academic year...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What does Volume 8 specify about including cli...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 states that clinical work conducted o...,single_hop_specifc_query_synthesizer
3,Is Title IV a term or non-term program?,[Non-Term Characteristics A program that measu...,"According to the context, Title IV programs ar...",single_hop_specifc_query_synthesizer
4,How do the payment periods relate to the requi...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The payment periods are linked to the academic...,multi_hop_abstract_query_synthesizer
5,how disbursmnt timing in subscrptn-based progr...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that for the first two su...,multi_hop_abstract_query_synthesizer
6,How do the guidelines and exceptions for clini...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines specify that clinical work cond...,multi_hop_abstract_query_synthesizer
7,Considering the participation and requirements...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Measuring progress in clock hours causes a pro...,multi_hop_abstract_query_synthesizer
8,How do the definitions and requirements outlin...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulations in Volume 2 specify that an ac...,multi_hop_specific_query_synthesizer
9,where appendix A and B show disbursement timin...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance and exam...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [32]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (also called student Federal PLUS Loans or parent PLUS Loans)\n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before it ended July 1, 2010)\n- Federal SLS Loans (also made under the FFEL Program before July 1, 2010)\n- Federal PLUS Loans (also made under the FFEL Program before July 1, 2010)\n\nAdditionally, there are Direct Consolidation Loans or Federal Consolidation Loans for borrowers who consolidate previous loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [33]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### Answer

qa evaluator: evaluates question answer quality 

## LangSmith Evaluation

In [35]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'passionate-thumb-48' at:
https://smith.langchain.com/o/e60ce1c7-6903-49a6-9264-5c7bd472b609/datasets/d24cb98d-d826-4a47-b009-5657c937811e/compare?selectedSessions=aa3f31ff-27f3-485d-95c0-d5ca95d52059




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the inclusion of clinical work in a s...,The inclusion of clinical work in a standard t...,None,"According to Volume 8, clinical work conducted...",1,1,0,4.240797,8968ec86-0717-477e-af81-46454289711d,9100fdbf-6efa-4e8e-8467-971cfa3fb0c6
1,Volume 2 and Volume 8 what they say about clin...,Based on the provided context:\n\n- **Volume 2...,None,Volume 2 explains that clinical work conducted...,1,1,0,10.359624,584f7793-6354-433c-911f-43730a197df4,e79752af-ea1e-4bcb-9a00-a4acf0f29820
2,where appendix A and B show disbursement timin...,Appendix B provides detailed guidance on the d...,None,Appendix B provides detailed guidance and exam...,1,1,0,3.395603,8d364a02-b428-47ea-b471-a18ffc60a201,3a06ad78-2bc4-427f-bf99-0151b56d2e03
3,How do the definitions and requirements outlin...,Based on the provided context:\n\nThe definiti...,None,The regulations in Volume 2 specify that an ac...,1,1,0,6.844610,e808e069-ba6a-4749-9278-484acf13aa79,67fc02fd-13cf-4338-8143-275d59177df2
4,Considering the participation and requirements...,Measuring progress in clock hours always resul...,None,Measuring progress in clock hours causes a pro...,1,1,0,5.237756,6088a0f8-7f26-4c59-affb-286f92dd5bed,5274b5ba-7e3d-4acc-a9c5-a7050cb8bd91
5,How do the guidelines and exceptions for clini...,"Based on the provided context, the guidelines ...",None,The guidelines specify that clinical work cond...,1,0,0,5.448275,24983ca5-fb7b-4237-bd97-6a24faa1f87d,575fc7ba-0d43-44c9-86d5-d329c541af89
6,how disbursmnt timing in subscrptn-based progr...,Based on the provided context:\n\nIn subscript...,None,The context explains that for the first two su...,1,1,0,6.332688,27d10777-6a2e-4c41-8492-067ab1954fdd,cedb3d58-df21-473f-93e0-8cd4a4e1c58b
7,How do the payment periods relate to the requi...,Payment periods for different programs are def...,None,The payment periods are linked to the academic...,1,1,0,6.538941,a96fea42-6df1-4964-9088-9141887d2eed,4758931c-e747-4199-a9bb-3c38b1b9308e
8,Is Title IV a term or non-term program?,"Based on the context, Title IV funds can be ap...",None,"According to the context, Title IV programs ar...",0,0,0,2.738412,db67f6ba-6ff7-48b6-b3f3-9cc3df888781,6a3574cd-f1c0-48c2-b0d0-a60618def475
9,What does Volume 8 specify about including cli...,"Volume 8, specifically Chapter 3, provides add...",None,Volume 8 states that clinical work conducted o...,0,0,0,2.318640,b7c64400-5853-4932-a759-645ecacae810,abad45cd-ba11-47e0-9250-5ef83c14d28f


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [36]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [62]:
rag_documents = docs

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [64]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [ ]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [66]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Agents? Man, they’re this super vague concept in the AI world. Some folks think of them as digital assistants that act on your behalf—like a travel agent. Others see them as AI models with tools, looping through tasks to solve problems. But here’s the kicker: the term is so fuzzy that it leaves you scratching your head, since everyone seems to have their own take on what it really means. Plus, there’s this whole issue of gullibility—how can these agents make smart choices if they can’t tell reality from fiction? So yeah, they’re like this elusive dream still waiting for a real breakthrough. 💫'

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'artistic-airplane-98' at:
https://smith.langchain.com/o/117cfda3-8a09-4ba4-9922-07b45fd73803/datasets/25fa804e-0ce3-4848-9cd1-c83d91988e78/compare?selectedSessions=42ea0e06-22a8-4bfb-a5bc-760a0cf4d280




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the introduction of GPT-4o impacted th...,"Yo, the rollout of GPT-4o is such a game chang...",None,The introduction of GPT-4o has significantly i...,1,1,1,4.590299,e749dc1c-f46a-4e61-9742-daa3d275671c,dffab390-a87b-4d08-93a3-8eedc7d24a7e
1,How did the breaking of the GPT-4 barrier in 2...,"Yo, the breaking of the GPT-4 barrier in 2024 ...",None,"In 2024, the breaking of the GPT-4 barrier sig...",1,0,1,3.217965,e0f5c6c4-fb8e-411e-8399-95212616c92b,d10800e7-981e-4127-8641-d2f357f52723
2,How does the training efficiency of DeepSeek v...,"Yo, check it! DeepSeek v3 is flexing some seri...",None,"DeepSeek v3, a 685B parameter model, is one of...",1,1,1,7.265255,28816b79-8c83-49b7-90f3-dc5a99ee6a7f,cddb24bd-3a3c-4ad3-9f10-cebb03122d05
3,How has the development of models surpassing G...,"Yo, the scene in 2024 is pretty wild! With 18 ...",None,"In 2024, the development of models surpassing ...",1,0,1,8.281109,757f3ec9-c32f-4d24-a2f7-d02b88fdb499,2fa50d79-687c-4902-a08c-9e603a4bab64
4,How do the ethics of AI and the gullibility of...,"Yo, the ethics of AI and the gullibility of la...",None,The ethics of AI and the gullibility of langua...,1,0,1,5.492535,bf0ac2b1-9738-4e30-8f8b-1b5d9e8c3b04,56695a52-31b0-4ce8-bd7f-f071a3649b82
5,What are the ethical concerns related to the g...,"Yo, the ethical vibes around the gullibility o...",None,The ethical concerns related to the gullibilit...,1,0,1,5.733880,d312a7f8-fd86-4df7-9d4d-6f9387a11412,32fe5cb2-0df4-4265-ad76-0487a7cbcb9f
6,Why are Large Language Models (LLMs) considere...,"LLMs are considered black boxes because, despi...",None,Large Language Models (LLMs) are considered bl...,1,1,1,7.673780,28659332-3637-4cee-9018-98d587575f5b,6d4eece2-36ce-45e1-b308-11ab50b14f34
7,What are the ethical concerns associated with ...,"Yo, the ethical vibes surrounding the gullibil...",None,The ethical concerns associated with the gulli...,1,1,1,15.375634,2da5c189-eb9e-419b-9c93-e3247d9f154c,6b2701ac-8dd9-42bc-b4cf-92c410f43390
8,What role does Stanford Alpaca play in the dev...,I don't know.,None,Stanford Alpaca is associated with the acceler...,0,0,0,3.234988,1c5369b1-7b7e-476d-9d28-816ce1a360f6,4be48806-dee2-4810-9d3a-6a07e97d14ac
9,Whaat are the key highlights from Simon Willis...,"Yo, let’s break it down! In 2023, Simon Willis...",None,Simon Willison's Weblog highlights that 2023 w...,1,0,1,5.724408,012f7bdc-73f0-4ad8-8a30-e89edbb0a0e7,2c0cd907-d3ed-452c-8bac-e5b3368a1980


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.